In [1]:
from datascience import *
path_data = '../../assets/data/'
import numpy as np
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')
%matplotlib inline

# Amostragem e Distribuições Empíricas
Uma parte importante da ciência de dados consiste em tirar conclusões baseadas nos dados de amostras aleatórias. Para interpretar corretamente seus resultados, os cientistas de dados precisam primeiro entender exatamente o que são amostras aleatórias.

Neste capítulo, vamos examinar mais atentamente a amostragem, com atenção especial às propriedades de grandes amostras aleatórias.

Vamos começar tirando algumas amostras. Nossos exemplos são baseados no conjunto de dados [`top_movies_2017.csv`](https://inferentialthinking.com/chapters/07/1/Visualizing_Categorical_Distributions.html#grouping-categorical-data).

In [2]:
top1 = Table.read_table(path_data + 'top_movies_2017.csv')
top2 = top1.with_column('Row Index', np.arange(top1.num_rows))
top = top2.move_to_start('Row Index')

top.set_format(make_array(3, 4), NumberFormatter)

Row Index,Title,Studio,Gross,Gross (Adjusted),Year
0,Gone with the Wind,MGM,"198,676,459","1,796,176,700",1939
1,Star Wars,Fox,"460,998,007","1,583,483,200",1977
2,The Sound of Music,Fox,"158,671,368","1,266,072,700",1965
3,E.T.: The Extra-Terrestrial,Universal,"435,110,554","1,261,085,000",1982
4,Titanic,Paramount,"658,672,302","1,204,368,000",1997
5,The Ten Commandments,Paramount,"65,500,000","1,164,590,000",1956
6,Jaws,Universal,"260,000,000","1,138,620,700",1975
7,Doctor Zhivago,MGM,"111,721,910","1,103,564,200",1965
8,The Exorcist,Warner Brothers,"232,906,145","983,226,600",1973
9,Snow White and the Seven Dwarves,Disney,"184,925,486","969,010,000",1937


<h2>Amostragem de Linhas de uma Tabela</h2>

Cada linha de uma tabela de dados representa um indivíduo; em `top`, cada indivíduo é um filme. A amostragem de indivíduos pode, portanto, ser realizada pela amostragem das linhas de uma tabela.

O conteúdo de uma linha são os valores de diferentes variáveis medidas no mesmo indivíduo. Portanto, os conteúdos das linhas amostradas formam amostras de valores de cada uma das variáveis.

<h2>Amostras Determinísticas</h2>

Quando você simplesmente especifica quais elementos de um conjunto deseja escolher, sem envolver quaisquer chances, você cria uma *amostra determinística*.

Você já fez isso muitas vezes, por exemplo, usando `take`:

In [3]:
top.take(make_array(3, 18, 100))

Row Index,Title,Studio,Gross,Gross (Adjusted),Year
3,E.T.: The Extra-Terrestrial,Universal,"435,110,554","1,261,085,000",1982
18,The Lion King,Buena Vista,"422,783,777","792,511,700",1994
100,The Hunger Games,Lionsgate,"408,010,692","452,174,400",2012


You have also used `where`:

In [4]:
top.where('Title', are.containing('Harry Potter'))

Row Index,Title,Studio,Gross,Gross (Adjusted),Year
74,Harry Potter and the Sorcerer's Stone,Warner Brothers,"317,575,550","497,066,400",2001
114,Harry Potter and the Deathly Hallows Part 2,Warner Brothers,"381,011,219","426,630,300",2011
131,Harry Potter and the Goblet of Fire,Warner Brothers,"290,013,036","401,608,200",2005
133,Harry Potter and the Chamber of Secrets,Warner Brothers,"261,988,482","399,302,200",2002
154,Harry Potter and the Order of the Phoenix,Warner Brothers,"292,004,738","377,314,200",2007
175,Harry Potter and the Half-Blood Prince,Warner Brothers,"301,959,197","359,788,300",2009
177,Harry Potter and the Prisoner of Azkaban,Warner Brothers,"249,541,069","357,233,500",2004


Embora sejam amostras, não são amostras aleatórias. Não envolvem acaso.

<h2>Amostras de Probabilidade</h2>

Para descrever amostras aleatórias, alguma terminologia será útil.

Uma *população* é o conjunto de todos os elementos dos quais uma amostra será retirada.

Uma *amostra probabilística* é aquela para a qual é possível calcular, antes de ser retirada, a probabilidade com que qualquer subconjunto de elementos entrará na amostra.

Em uma amostra probabilística, todos os elementos não precisam ter a mesma chance de serem escolhidos.

<h2>Um Esquema de Amostragem Aleatória</h2>

Por exemplo, suponha que você escolha duas pessoas de uma população que consiste em três pessoas A, B e C, de acordo com o seguinte esquema:

- A pessoa A é escolhida com probabilidade 1.
- Uma das pessoas B ou C é escolhida de acordo com o lançamento de uma moeda: se a moeda cair em cara, você escolhe B, e se cair em coroa você escolhe C.

Esta é uma amostra probabilística de tamanho 2. Aqui estão as chances de entrada para todos os subconjuntos não vazios:

    A: 1 
    B: 1/2
    C: 1/2
    AB: 1/2
    AC: 1/2
    BC: 0
    ABC: 0

A pessoa A tem uma chance maior de ser selecionada do que as pessoas B ou C; de fato, a pessoa A certamente será selecionada. Como essas diferenças são conhecidas e quantificadas, elas podem ser levadas em conta ao trabalhar com a amostra.

<h2>Uma Amostra Sistemática</h2>

Imagine todos os elementos da população listados em uma sequência. Um método de amostragem começa escolhendo uma posição aleatória no início da lista e, em seguida, posições uniformemente espaçadas depois disso. A amostra consiste nos elementos dessas posições. Tal amostra é chamada de *amostra sistemática*.

Aqui escolheremos uma amostra sistemática das linhas de `top`. Começaremos escolhendo uma das primeiras 10 linhas aleatoriamente e, em seguida, escolheremos a cada 10ª linha após isso.

In [5]:
"""Escolha um início aleatório entre as linhas 0 a 9;
então pegue cada 10ª linha."""

start = np.random.choice(np.arange(10))
top.take(np.arange(start, top.num_rows, 10))

Row Index,Title,Studio,Gross,Gross (Adjusted),Year
6,Jaws,Universal,"260,000,000","1,138,620,700",1975
16,Jurassic Park,Universal,"402,453,882","817,186,200",1993
26,Mary Poppins,Disney,"102,272,727","695,036,400",1964
36,Love Story,Paramount,"106,397,186","622,283,500",1970
46,The Robe,Fox,"36,000,000","581,890,900",1953
56,Rogue One: A Star Wars Story,Buena Vista,"532,177,324","537,326,000",2016
66,The Dark Knight Rises,Warner Brothers,"448,139,099","511,902,300",2012
76,Close Encounters of the Third Kind,Columbia,"132,088,635","494,066,600",1977
86,Transformers: Revenge of the Fallen,Paramount/Dreamworks,"402,111,870","479,179,200",2009
96,Toy Story 3,Buena Vista,"415,004,880","464,074,600",2010


Execute a célula algumas vezes para ver como a saída varia. 

Esta amostra sistemática é uma amostra probabilística. Neste esquema, todas as linhas têm chance $1/10$ de serem escolhidas. Por exemplo, a Linha 23 é escolhida se e somente se a Linha 3 for escolhida, e a chance disso é $1/10$. 

Mas nem todos os subconjuntos têm a mesma chance de serem escolhidos. Como as linhas selecionadas estão espaçadas uniformemente, a maioria dos subconjuntos de linhas não tem chance de serem escolhidos. Os únicos subconjuntos que são possíveis são aqueles que consistem em linhas separadas por múltiplos de 10. Qualquer um desses subconjuntos é selecionado com chance de 1/10. Outros subconjuntos, como um subconjunto contendo tanto a 15ª quanto a 16ª linha da tabela, ou qualquer subconjunto de tamanho superior a 10, são selecionados com chance 0.

<h2>Amostras Aleatórias Com ou Sem Reposição</h2>
 
Neste curso, lidaremos principalmente com os dois métodos mais diretos de amostragem. 

O primeiro é a amostragem aleatória com reposição, que (como vimos anteriormente) é o comportamento padrão de `np.random.choice` ao amostrar de uma matriz. 

O outro, chamado de "amostra aleatória simples", é uma amostra retirada aleatoriamente *sem* reposição. Indivíduos amostrados não são substituídos na população antes que o próximo indivíduo seja selecionado. Este é o tipo de amostragem que ocorre ao lidar uma mão de um baralho de cartas, por exemplo. Para usar `np.random.choice` para amostragem aleatória simples, você deve incluir o argumento `replace=False`.

Neste capítulo, usaremos simulação para estudar o comportamento de amostras grandes retiradas aleatoriamente com ou sem reposição.

<h2>Amostras de Conveniência</h2>
Desenhar uma amostra aleatória requer cuidado e precisão. Não é aleatório, embora esse seja um significado coloquial da palavra "aleatório". Se você ficar em uma esquina e tomar como amostra as primeiras dez pessoas que passam, você pode pensar que está fazendo uma amostragem aleatória porque não escolheu quem passou por ali. Mas não é uma amostra aleatória – é uma *amostra de conveniência*. Você não sabia antecipadamente a probabilidade de cada pessoa entrar na amostra; talvez você nem tenha especificado exatamente quem estava na população.